1. 导入必要的库

In [362]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
from scipy.spatial.transform import Rotation
import time

2. DH参数变换矩阵计算

In [363]:

def dh_transform(a, alpha, d, theta):
    """
    计算DH变换矩阵
    
    参数:
    a: 连杆长度
    alpha: 连杆扭角(弧度)
    d: 连杆偏距
    theta: 关节角度(弧度)
    
    返回:
    T: 4x4的DH变换矩阵
    """
    # 构建DH变换矩阵
    ct = np.cos(theta)
    st = np.sin(theta)
    ca = np.cos(alpha)
    sa = np.sin(alpha)
    
    T = np.array([
        [ct, -st, 0, a],
        [st*ca, ct*ca, -sa, -d*sa],
        [st*sa, ct*sa, ca, d*ca],
        [0, 0, 0, 1]
    ])
    
    return T

3. 前向运动学计算函数

In [364]:
def forward_kinematics(angles, dh_params):
    """
    计算6轴机器人正向运动学
    
    参数:
        angles: 关节角度 [theta1, theta2, theta3, theta4, theta5, theta6]
        dh_params: DH参数字典
        
    返回:
        T: 末端执行器相对于基座的变换矩阵，4x4 np.array
    """
    theta1, theta2, theta3, theta4, theta5, theta6 = angles
    
    # 提取DH参数
    a0 = dh_params.get('a0', 0)
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    a3 = dh_params.get('a3', 0)
    a4 = dh_params.get('a4', 0)
    a5 = dh_params.get('a5', 0)
    
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    d4 = dh_params.get('d4', 0)
    d5 = dh_params.get('d5', 0)
    d6 = dh_params.get('d6', 0)
    
    alpha0 = dh_params.get('alpha0', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    alpha3 = dh_params.get('alpha3', 0)
    alpha4 = dh_params.get('alpha4', 0)
    alpha5 = dh_params.get('alpha5', 0)
    
    # 计算各个变换矩阵
    T0_1 = dh_transform(a0, alpha0, d1, theta1)
    T1_2 = dh_transform(a1, alpha1, d2, theta2-np.pi/2)  # 第二个关节偏移90度
    T2_3 = dh_transform(a2, alpha2, d3, theta3)
    T3_4 = dh_transform(a3, alpha3, d4, theta4)
    T4_5 = dh_transform(a4, alpha4, d5, theta5)
    T5_6 = dh_transform(a5, alpha5, d6, theta6+np.pi) #  第六个关节偏移180度
    
    # 计算总变换矩阵
    T0_2 = np.dot(T0_1, T1_2)
    T0_3 = np.dot(T0_2, T2_3)
    T0_4 = np.dot(T0_3, T3_4)
    T0_5 = np.dot(T0_4, T4_5)
    T0_6 = np.dot(T0_5, T5_6)
     
    return T0_6

def forward_kinematics_first3joints(angles, dh_params):
    """
    计算前3个关节的正向运动学，用于LM算法中的关节1-3求解
    
    参数:
        angles: 关节角度 [theta1, theta2, theta3]
        dh_params: DH参数字典
        
    返回:
        T: 第3个关节坐标系相对于基座的变换矩阵，4x4 np.array
    """
    theta1, theta2, theta3 = angles
    
    # 提取DH参数
    a0 = dh_params.get('a0', 0)
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    
    alpha0 = dh_params.get('alpha0', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    
    # 计算各个变换矩阵
    T0_1 = dh_transform(a0, alpha0, d1, theta1)
    T1_2 = dh_transform(a1, alpha1, d2, theta2-np.pi/2)  # 第二个关节偏移90度
    T2_3 = dh_transform(a2, alpha2, d3, theta3)
    
    # 计算总变换矩阵
    T0_2 = np.dot(T0_1, T1_2)
    T0_3 = np.dot(T0_2, T2_3)
    

    return T0_3

4. 位姿误差计算函数

In [365]:
def compute_pose_error(T_current, T_desired):
    """
    计算两个位姿之间的误差
    
    参数:
        T_current: 当前变换矩阵，4x4 np.array
        T_desired: 目标变换矩阵，4x4 np.array
    
    返回:
        error: 位姿误差向量，6x1 np.array (前3个元素是位置误差，后3个元素是姿态误差)
    """
    # 计算位置误差 (简单的欧几里得距离)
    pos_error = T_desired[:3, 3] - T_current[:3, 3]
    
    # 计算旋转误差 (使用轴角表示法)
    R_current = T_current[:3, :3]
    R_desired = T_desired[:3, :3]
    
    # 计算旋转误差矩阵 R_d * R_c^T
    R_error = R_desired @ R_current.T
    
    # 将旋转矩阵转换为轴角表示
    angle = np.arccos((np.trace(R_error) - 1) / 2)
    
    if abs(angle) < 1e-10:
        axis = np.zeros(3)
    else:
        axis =  np.array([
            R_error[2, 1] - R_error[1, 2],
            R_error[0, 2] - R_error[2, 0],
            R_error[1, 0] - R_error[0, 1]
        ]) / (2 * np.sin(angle))
    
    # 轴角表示：轴 * 角度
    rot_error = axis * angle
    
    # 组合位置和旋转误差
    return np.concatenate([pos_error, rot_error])

5. 自定义雅可比矩阵计算函数

In [366]:
def custom_jacobian(angles, dh_params):
    """
    计算残差函数对关节角度的雅可比矩阵
    
    参数:
        angles: 当前关节角度 [theta1, theta2, theta3]
        dh_params: DH参数字典
        
    返回:
        6×3的雅可比矩阵，每列代表对应关节的影响
    """
    theta1, theta2, theta3 = angles
    # 提取DH参数
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    
    # 计算各个变换矩阵
    T0_1 = dh_transform(0, 0, d1, theta1)
    T1_2 = dh_transform(a1, alpha1, d2, theta2-np.pi/2)  # 第二个关节有90度的偏移
    T2_3 = dh_transform(a2, alpha2, d3, theta3)
    
    # 计算T0_2和T0_3
    T0_2 = np.dot(T0_1, T1_2)
    T0_3_current = np.dot(T0_2, T2_3)
    
    # 计算各关节坐标系的z轴方向（旋转轴）
    z0 = np.array([0, 0, 1])  # 基坐标系的z轴
    z1 = T0_1[:3, 2]          # 关节1坐标系的z轴
    z2 = T0_2[:3, 2]          # 关节2坐标系的z轴
    
    # 计算各关节的位置
    o0 = np.array([0, 0, 0])
    o1 = T0_1[:3, 3]
    o2 = T0_2[:3, 3]
    o3 = T0_3_current[:3, 3]  # 末端位置
    
    # 计算位置雅可比矩阵 (3×3)
    J_v = np.zeros((3, 3))
    
    # 关节1对末端位置的影响
    J_v[:, 0] = np.cross(z0, o3 - o0)
    
    # 关节2对末端位置的影响
    J_v[:, 1] = np.cross(z1, o3 - o1)
    
    # 关节3对末端位置的影响
    J_v[:, 2] = np.cross(z2, o3 - o2)
    
    # 计算旋转雅可比矩阵 (3×3)
    J_w = np.zeros((3, 3))
    J_w[:, 0] = z0  # 关节1的旋转轴
    J_w[:, 1] = z1  # 关节2的旋转轴
    J_w[:, 2] = z2  # 关节3的旋转轴
    
    # 合并为完整的雅可比矩阵 (6×3)
    J = np.vstack([J_v, J_w])
    
    return J

6. 数值雅可比矩阵计算函数（可选）

In [367]:
def numerical_jacobian(angles, dh_params, target_pose, epsilon=1e-6):
    """
    使用数值差分法计算雅可比矩阵
    
    参数:
        angles: 当前关节角度 [theta1, theta2, theta3]
        dh_params: DH参数字典
        target_pose: 目标位姿矩阵
        epsilon: 差分步长
        
    返回:
        6×3的雅可比矩阵，每列代表对应关节的影响
    """
    # 初始化雅可比矩阵
    jac = np.zeros((6, 3))
    
    # 计算当前位姿
    T_current = forward_kinematics_first3joints(angles, dh_params)
    
    # 计算当前误差
    error = compute_pose_error(T_current, target_pose)
    
    # 对每个关节计算偏导数
    for i in range(3):
        # 创建扰动角度
        angles_plus = angles.copy()
        angles_plus[i] += epsilon
        
        # 计算扰动后的位姿
        T_plus = forward_kinematics_first3joints(angles_plus, dh_params)
        
        # 计算扰动后的误差
        error_plus = compute_pose_error(T_plus, target_pose)
        
        # 计算雅可比矩阵的一列（误差对角度的导数）
        jac[:, i] = (error_plus - error) / epsilon
    
    return jac

7. Levenberg-Marquardt 算法实现

In [ ]:
class LMSolver:
    """
    完全按照KDL实现的Levenberg-Marquardt算法求解器
    """
    def __init__(self, dh_params):
        """
        初始化LM求解器
        
        参数:
            dh_params: DH参数字典
        """
        self.dh_params = dh_params
        self.nj = 3  # 我们求解前3个关节
        
        # 设置算法参数
        self.maxiter = 50000      # 最大迭代次数
        self.eps = 1e-12          # 位姿误差收敛阈值
        self.eps_joints = 1e-15   # 关节角度收敛阈值
        
        # 权重参数 - 可以调整位置和旋转误差的权重
        self.L = np.ones(6)
        self.L[:3] = 10         # 位置误差权重
        self.L[3:] = 0.1       # 姿态误差权重
        
        # 结果信息
        self.lastNrOfIter = 0
        self.lastDifference = 0
        self.lastTransDiff = 0
        self.lastRotDiff = 0
        self.lastSV = None
        
        self.display_information = True  # 是否显示迭代信息
        
        self.enable_early_termination = True # 是否启用提前终止
    
    def cartToJnt(self, q_init, target_pose, jac_method='custom'):
        """
        使用LM算法求解逆运动学
        
        参数:
            q_init: 初始关节角度, 长度为nj的np.array
            target_pose: 目标位姿(4x4变换矩阵)
            jac_method: 雅可比矩阵计算方法 ('custom' 或 'numerical')
        
        返回:
            q_out: 求解得到的关节角度
            error: 错误码
            history: 迭代历史数据
        """
        # 检查参数尺寸
        if len(q_init) != self.nj:
            return q_init, self.E_SIZE_MISMATCH, None
        
        # 初始化历史记录
        history = {
            'iterations': [],
            'q': [q_init.copy()],
            'errors': []
        }
        
        # 初始化基本参数 (按照KDL实现)
        v = 2.0         # 步长因子
        tau = 10.0      # 初始阻尼因子系数
        
        # 复制初始关节角度
        q = q_init.copy()
        q_out = q.copy()  # 初始化输出变量
        
        # 计算初始位姿 - 对应compute_fwdpos(q)
        T_base_head = forward_kinematics_first3joints(q, self.dh_params)
        
        # 计算初始误差 - 对应Twist_to_Eigen(diff(...))
        delta_pos = compute_pose_error(T_base_head, target_pose)
        
        # 应用权重 - 对应L.asDiagonal()*delta_pos
        delta_pos = self.L * delta_pos
        
        # 计算误差范数 - 对应delta_pos.norm()
        delta_pos_norm = np.linalg.norm(delta_pos)
        
        # 检查初始误差是否已经足够小
        if delta_pos_norm < self.eps:
            self.lastNrOfIter = 0
            
            # 重新计算未加权的误差 - 对应Twist_to_Eigen(diff(...))
            delta_pos_unweighted = compute_pose_error(T_base_head, target_pose)
            self.lastDifference = np.linalg.norm(delta_pos_unweighted)
            self.lastTransDiff = np.linalg.norm(delta_pos_unweighted[:3])
            self.lastRotDiff = np.linalg.norm(delta_pos_unweighted[3:])
            
            # 计算雅可比矩阵
            if jac_method == 'custom':
                jac = custom_jacobian(q, self.dh_params)
            else:  # numerical
                jac = numerical_jacobian(q, self.dh_params, target_pose)
            
            jac_weighted = np.diag(self.L) @ jac
            
            # 计算SVD
            _, s, _ = np.linalg.svd(jac_weighted, full_matrices=False)
            self.lastSV = s
            
            return q, self.E_NOERROR, history
        
        # 计算雅可比矩阵 - 对应compute_jacobian(q)
        if jac_method == 'custom':
            jac = custom_jacobian(q, self.dh_params)
        else:  # numerical
            jac = numerical_jacobian(q, self.dh_params, target_pose)
        
        # 应用权重到雅可比矩阵 - 对应jac = L.asDiagonal()*jac
        jac = np.diag(self.L) @ jac
        
        # 初始化lambda (按照KDL实现)
        lambda_ = tau
        
        # 主迭代循环
        for i in range(self.maxiter):
            # 计算SVD (每次迭代都重新计算) - 对应svd.compute(jac)
            U, s, Vh = np.linalg.svd(jac, full_matrices=False)
            
            # 修改奇异值 - 对应修改original_Aii
            original_Aii = np.zeros_like(s)
            for j in range(len(s)):  # 对所有奇异值应用
                original_Aii[j] = s[j] / (s[j]*s[j] + lambda_)
            
            # 计算关节角增量 - 对应KDL中的Segment A部分
            tmp = U.T @ delta_pos
            tmp = tmp * original_Aii
            diffq = Vh.T @ tmp
            
            # 计算梯度 - 对应grad = jac.transpose()*delta_pos
            grad = jac.T @ delta_pos
            
            # 输出迭代信息
            if self.display_information and (i % 100 == 0 or i < 5):
                print(f"------- iteration {i} ----------------")
                print(f"  q              = {q}")
                print(f"  lambda         = {lambda_}")
                print(f"  eigenvalues    = {s}")
                print(f"  difference norm= {delta_pos_norm}")
                print(f"  grad norm      = {np.linalg.norm(grad)}")
                print("")
            
            # 记录历史
            history['iterations'].append(i)
            history['q'].append(q.copy())
            history['errors'].append(delta_pos_norm)
            
            # 检查步长是否足够小 - 对应dnorm = diffq.lpNorm<Eigen::Infinity>()
            dnorm = np.max(np.abs(diffq))  # 无穷范数
            if self.enable_early_termination and dnorm < self.eps_joints:
                self.lastDifference = delta_pos_norm
                self.lastNrOfIter = i
                self.lastSV = s
                q_out = q.copy()
                
                # 重新计算当前位姿和误差
                T_base_head = forward_kinematics_first3joints(q, self.dh_params)
                delta_pos_unweighted = compute_pose_error(T_base_head, target_pose)
                self.lastTransDiff = np.linalg.norm(delta_pos_unweighted[:3])
                self.lastRotDiff = np.linalg.norm(delta_pos_unweighted[3:])
                print(f"步长过小，停止迭代,迭代次数：{i}")
                return q_out, history
            
            # 检查梯度是否足够小 - 对应grad.transpose()*grad < eps_joints*eps_joints
            if self.enable_early_termination and np.dot(grad, grad) < self.eps_joints * self.eps_joints:
                # 重新计算当前位姿和误差
                T_base_head = forward_kinematics_first3joints(q, self.dh_params)
                delta_pos_unweighted = compute_pose_error(T_base_head, target_pose)
                
                self.lastDifference = np.linalg.norm(delta_pos_unweighted)
                self.lastTransDiff = np.linalg.norm(delta_pos_unweighted[:3])
                self.lastRotDiff = np.linalg.norm(delta_pos_unweighted[3:])
                self.lastSV = s
                self.lastNrOfIter = i
                q_out = q.copy()
                print(f"梯度过小，停止迭代,迭代次数：{i}")
                return q_out, history
            
            # 计算新的关节角度 - 对应q_new = q+diffq
            q_new = q + diffq
            print(f"theta1 = {np.degrees(q_new[0]):.2f}°")
            print(f"theta2 = {np.degrees(q_new[1]):.2f}°")
            print(f"theta3 = {np.degrees(q_new[2]):.2f}°")
            # 计算新的位姿和误差
            T_base_head_new = forward_kinematics_first3joints(q_new, self.dh_params)
            delta_pos_new = compute_pose_error(T_base_head_new, target_pose)
            delta_pos_new = self.L * delta_pos_new  # 应用权重
            delta_pos_new_norm = np.linalg.norm(delta_pos_new)
            
            # 计算信任区域比率 - 对应KDL中的rho计算
            rho = delta_pos_norm**2 - delta_pos_new_norm**2
            denominator = diffq @ (lambda_ * diffq + grad)
            
            # 防止除零
            if abs(denominator) < 1e-10:
                if rho > 0:
                    rho = 1.0
                else:
                    rho = -1.0
            else:
                rho = rho / denominator
            
            # 根据信任区域比率更新参数
            if rho > 0:
                # 更新成功，接受新的关节角度
                q = q_new.copy()
                delta_pos = delta_pos_new.copy()
                delta_pos_norm = delta_pos_new_norm
                
                # 检查是否收敛
                if self.enable_early_termination and delta_pos_norm < self.eps:
                    # 重新计算未加权的误差
                    delta_pos_unweighted = compute_pose_error(T_base_head_new, target_pose)
                    self.lastDifference = np.linalg.norm(delta_pos_unweighted)
                    self.lastTransDiff = np.linalg.norm(delta_pos_unweighted[:3])
                    self.lastRotDiff = np.linalg.norm(delta_pos_unweighted[3:])
                    self.lastSV = s
                    self.lastNrOfIter = i
                    q_out = q.copy()
                    print(f"收敛，停止迭代,迭代次数：{i}")
                    return q_out, self.E_NOERROR, history
                
                # 重新计算雅可比矩阵 - 使用新的关节角度
                if jac_method == 'custom':
                    jac = custom_jacobian(q, self.dh_params)
                else:  # numerical
                    jac = numerical_jacobian(q, self.dh_params, target_pose)
                
                jac = np.diag(self.L) @ jac  # 应用权重
                # 更激进地减小lambda
                if rho > 0.75:  # 如果进展很好
                    lambda_ = lambda_ / 5.0
                elif rho > 0.25:  # 如果进展中等
                    lambda_ = lambda_ / 2.0
                else:  # 进展较小但仍为正
                    lambda_ = lambda_ * 0.9
                # 减小阻尼因子 (KDL方式) - 对应lambda = lambda*max(1/3.0, 1-tmp*tmp*tmp)
                tmp = 2 * rho - 1
                lambda_ = lambda_ * max(1/3.0, 1 - tmp**3)
                v = 2
            else:
                # 更新失败，增大阻尼因子 - 对应lambda = lambda*v; v = 2*v
                lambda_ = lambda_ * v
                v = 2 * v
        
        # 达到最大迭代次数
        self.lastDifference = delta_pos_norm
        self.lastTransDiff = np.linalg.norm(delta_pos[:3] / self.L[:3])
        self.lastRotDiff = np.linalg.norm(delta_pos[3:] / self.L[3:])
        self.lastSV = s
        self.lastNrOfIter = self.maxiter
        q_out = q.copy()
        
        if self.display_information:
            print(f"达到最大迭代次数 {self.maxiter}")
            print(f"最终位置误差: {self.lastTransDiff}")
            print(f"最终旋转误差: {self.lastRotDiff}")
        
        return q_out, history

8. 实际T03计算函数

In [369]:
def calculate_wrist_point(tcp_pose, theta4, theta5, theta6, dh_params):
    """
    
    参数:
        tcp_pose: TCP位姿矩阵 (4x4)
        theta4, theta5, theta6: 腕关节角度 (弧度)
        dh_params: DH参数字典
        
    返回:
        wrist_pose: 腕点位姿矩阵 (4x4)
    """
    # 提取相关DH参数
    a3 = dh_params.get('a3', 0)
    a4 = dh_params.get('a4', 0)
    a5 = dh_params.get('a5', 0)
    
    d4 = dh_params.get('d4', 0)
    d5 = dh_params.get('d5', 0)
    d6 = dh_params.get('d6', 0)
    
    alpha3 = dh_params.get('alpha3', 0)
    alpha4 = dh_params.get('alpha4', 0)
    alpha5 = dh_params.get('alpha5', 0)
    
    # 计算腕部变换矩阵
    T3_4 = dh_transform(a3, alpha3, d4, theta4)
    T4_5 = dh_transform(a4, alpha4, d5, theta5)
    T5_6 = dh_transform(a5, alpha5, d6, theta6+np.pi)  # 腕部末端偏移180度
    
    # 计算腕部到TCP的变换矩阵
    T3_6 = np.dot(T3_4, np.dot(T4_5, T5_6))
    
    # 计算腕点位姿
    wrist_pose = np.dot(tcp_pose, np.linalg.inv(T3_6))
    
    return wrist_pose

9. 完整的逆运动学求解函数

In [370]:
def solve_inverse_kinematics_lm(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                              theta4, theta5, theta6, dh_params, 
                              initial_guess=None, jac_method='custom'):
    """
    使用LM算法求解6轴机器人逆运动学前3个关节角度
    
    参数:
        x, y, z: TCP位置坐标
        nx, ny, nz: TCP位姿的x轴方向
        ox, oy, oz: TCP位姿的y轴方向
        ax, ay, az: TCP位姿的z轴方向
        theta4, theta5, theta6: 已知的关节角度(弧度)
        dh_params: 机器人DH参数字典
        initial_guess: 初始关节角度猜测值，默认为零位
        jac_method: 雅可比矩阵计算方法 ('custom' 或 'numerical')
        
    返回:
        theta1, theta2, theta3: 计算得到的前3个关节角度(弧度)
        error_code: 错误码
    """
    # 构建TCP位姿矩阵
    tcp_pose = np.array([
        [nx, ox, ax, x],
        [ny, oy, ay, y],
        [nz, oz, az, z],
        [0, 0, 0, 1]
    ])
    
    # 计算腕点位置 (从TCP向后推算)T03矩阵
    T03 = calculate_wrist_point(tcp_pose, theta4, theta5, theta6, dh_params)
    
    # 初始化LM求解器
    solver = LMSolver(dh_params)
    solver.display_information = False  # 是否显示迭代信息
    
    # 设置初始猜测值
    if initial_guess is None:
        initial_guess = np.zeros(3)  # 默认全部为0
    
    # 使用LM算法求解前3个关节角度
    q_sol,history= solver.cartToJnt(initial_guess, T03, jac_method)
    
    # 返回计算得到的关节角度
    theta1, theta2, theta3 = q_sol
    
    # 规范化角度到[-pi, pi]范围域
    theta1 = ((theta1 + np.pi) % (2*np.pi)) - np.pi
    theta2 = ((theta2 + np.pi) % (2*np.pi)) - np.pi
    theta3 = ((theta3 + np.pi) % (2*np.pi)) - np.pi
    return theta1, theta2, theta3

In [371]:
def solve_with_multiple_initial_guesses(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                                       theta4, theta5, theta6, dh_params, jac_method='custom'):
    """
    使用多个初始猜测值尝试求解逆运动学
    """
    initial_guesses = [
        [0.0, 0.0, 0.0],
        [np.pi/4, np.pi/4, np.pi/4],
        [-np.pi/4, np.pi/4, np.pi/4],
        [np.pi/4, -np.pi/4, np.pi/4],
        [np.pi/4, np.pi/4, -np.pi/4],
        [np.pi/2, 0.0, 0.0],
        [0.0, np.pi/2, 0.0],
        [0.0, 0.0, np.pi/2],
        [np.pi/3, np.pi/3, np.pi/3],
        [-np.pi/3, -np.pi/3, -np.pi/3],
        [np.pi/6, np.pi/6, np.pi/6],
        [-np.pi/6, -np.pi/6, -np.pi/6]
    ]
    
    # 跟踪最佳结果
    best_result = None
    best_error = float('inf')
    
    
    print("===== 尝试多个初始值 =====")
    
    # 尝试每个初始猜测值
    for i, guess in enumerate(initial_guesses):
        print(f"\n初始值 #{i+1}: θ1={np.degrees(guess[0]):.1f}°, θ2={np.degrees(guess[1]):.1f}°, θ3={np.degrees(guess[2]):.1f}°")
        
        try:
            # 求解逆运动学
            theta1, theta2, theta3= solve_inverse_kinematics_lm(
                x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                theta4, theta5, theta6, dh_params, 
                initial_guess=guess,
                jac_method=jac_method
            )
            
            # 验证结果
            joint_angles = [theta1, theta2, theta3, theta4, theta5, theta6]
            result_pose = forward_kinematics(joint_angles, dh_params)
            result_pos = result_pose[:3, 3]
            
            # 计算误差
            position_error = np.linalg.norm(result_pos - np.array([x, y, z]))
            
            # 输出结果
            print(f"  位置误差: {position_error:.2f} mm")
            print(f"  关节角度: θ1={np.degrees(theta1):.1f}°, θ2={np.degrees(theta2):.1f}°, θ3={np.degrees(theta3):.1f}°")
            
            # 检查是否是最佳结果
            if position_error < best_error:
                best_error = position_error
                best_result = (theta1, theta2, theta3)
           
        except Exception as e:
            print(f"  求解出错: {e}")
            continue
    
    
    # 输出最佳结果
    if best_result is not None:
        theta1, theta2, theta3 = best_result
        print(f"\n===== 最佳解 =====")
        print(f"关节角度: θ1={np.degrees(theta1):.1f}°, θ2={np.degrees(theta2):.1f}°, θ3={np.degrees(theta3):.1f}°")
        print(f"位置误差: {best_error:.4f} mm")
        
        return theta1, theta2, theta3
    else:
        print("\n未找到有效解!")
        return None, None, None

In [372]:

# TCP位置坐标和姿态  
x, y, z = 915.249, 608.436, 233.411  # 位置(mm)  

# 使用四元数表示姿态（也可以直接使用旋转矩阵）
from scipy.spatial.transform import Rotation  
quat = [0.3604, 0.8224, 0.3919, -0.2006]  # [x, y, z, w]

# 创建旋转对象
rot = Rotation.from_quat(quat)

# 获取旋转矩阵
rotation_matrix = rot.as_matrix() 

# 提取姿态矩阵的各列向量  
nx, ny, nz = rotation_matrix[:, 0]  # x轴方向向量
ox, oy, oz = rotation_matrix[:, 1]  # y轴方向向量  
ax, ay, az = rotation_matrix[:, 2]  # z轴方向向量

# 已知的关节4,5,6角度(弧度)  
theta4 = np.radians(90)  
theta5 = np.radians(45)  
theta6 = np.radians(0)  

# 设置机器人DH参数(工业机器人 NB12s-1214-5A)  
dh_params = {  
    'a0': 0,      # 连杆1长度(mm)
    'a1': 100,    # 连杆2长度(mm) 
    'a2': 680,    # 连杆3长度(mm)
    'a3': 50,     # 连杆4长度(mm)
    'a4': 0,      # 连杆5长度(mm)
    'a5': 0,      # 连杆6长度(mm)
    'd1': 500,    # 关节1偏移(mm)
    'd2': 0,      # 关节2偏移(mm)
    'd3': 0,      # 关节3偏移(mm
    'd4': 660,    # 关节4偏移(mm)
    'd5': 0,      # 关节5偏移(mm)
    'd6': 98,     # 关节6偏移(mm)，末端执行器偏移
    'alpha0': 0,          # 连杆0扭角(弧度)
    'alpha1': -np.pi/2,   # 连杆1扭角(弧度)
    'alpha2': 0,          # 连杆2扭角(弧度)
    'alpha3': -np.pi/2,   # 连杆3扭角(弧度)
    'alpha4': np.pi/2,    # 连杆4扭角(弧度)
    'alpha5': -np.pi/2,   # 连杆5扭角(弧度)
}

print("===== 使用LM算法求解6轴机器人逆运动学 =====")
print(f"目标TCP位置: [{x}, {y}, {z}] mm")
print(f"目标TCP姿态: nx={nx:.3f}, ny={ny:.3f}, nz={nz:.3f}, ox={ox:.3f}, oy={oy:.3f}, oz={oz:.3f}, ax={ax:.3f}, ay={ay:.3f}, az={az:.3f}")

# 构建TCP位姿矩阵
tcp_pose = np.array([
    [nx, ox, ax, x],
    [ny, oy, ay, y],
    [nz, oz, az, z],
    [0, 0, 0, 1]
])

print("TCP位姿矩阵:")
print(tcp_pose)
# 开始计时
start_time = time.time()

# 使用自定义雅可比矩阵的LM算法求解
print("\n使用自定义雅可比矩阵的LM算法求解...")
theta1, theta2, theta3 = solve_with_multiple_initial_guesses(
    x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
    theta4, theta5, theta6, dh_params, 
    jac_method='custom'
)

# 计算求解时间
elapsed_time = time.time() - start_time
print(f"求解时间: {elapsed_time:.4f} 秒")

# 显示求解结果
print("\n求解结果:")
print(f"关节1角度: {np.degrees(theta1):.2f}°")
print(f"关节2角度: {np.degrees(theta2):.2f}°")
print(f"关节3角度: {np.degrees(theta3):.2f}°")

#S实际T03矩阵
T03_pre = calculate_wrist_point(tcp_pose, theta4, theta5, theta6, dh_params)
print("\n实际T03矩阵:")
print(T03_pre)
#计算T03
T03= forward_kinematics_first3joints([theta1, theta2, theta3], dh_params)
print("\n计算T03矩阵:")
print(T03)

# 验证求解结果
print("\n验证求解结果:")

# 所有关节角度
joint_angles = [theta1, theta2, theta3, theta4, theta5, theta6]

# 使用正向运动学验证
result_pose = forward_kinematics(joint_angles, dh_params)
result_pos = result_pose[:3, 3]

# 计算位置误差
position_error = np.linalg.norm(result_pos - np.array([x, y, z]))

print(f"原始TCP位置: [{x}, {y}, {z}] mm")
print(f"验证TCP位置: [{result_pos[0]:.2f}, {result_pos[1]:.2f}, {result_pos[2]:.2f}] mm")
print(f"位置误差: {position_error:.2f} mm")


# 比较姿态矩阵
result_rot = result_pose[:3, :3]
rotation_error = np.linalg.norm(rotation_matrix - result_rot)
print(f"姿态误差: {rotation_error:.6f}")

# 打印所有关节角度
print("\n完整关节角度:")
print(f"关节1: {np.degrees(theta1):.2f}°")
print(f"关节2: {np.degrees(theta2):.2f}°")
print(f"关节3: {np.degrees(theta3):.2f}°")
print(f"关节4: {np.degrees(theta4):.2f}°")
print(f"关节5: {np.degrees(theta5):.2f}°")
print(f"关节6: {np.degrees(theta6):.2f}°")


===== 使用LM算法求解6轴机器人逆运动学 =====
目标TCP位置: [915.249, 608.436, 233.411] mm
目标TCP姿态: nx=-0.660, ny=0.436, nz=0.612, ox=0.750, oy=0.433, oz=0.500, ax=-0.047, ay=0.789, az=-0.612
TCP位姿矩阵:
[[-6.59761976e-01  7.49974284e-01 -4.74627073e-02  9.15249000e+02]
 [ 4.35531298e-01  4.33084145e-01  7.89145495e-01  6.08436000e+02]
 [ 6.12394173e-01  4.99976696e-01 -6.12369725e-01  2.33411000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

使用自定义雅可比矩阵的LM算法求解...
===== 尝试多个初始值 =====

初始值 #1: θ1=0.0°, θ2=0.0°, θ3=0.0°
theta1 = 98.67°
theta2 = 98.67°
theta3 = 41.85°
theta1 = 98.67°
theta2 = 98.67°
theta3 = 41.85°
theta1 = 98.67°
theta2 = 98.67°
theta3 = 41.85°
theta1 = 98.67°
theta2 = 98.67°
theta3 = 41.85°
theta1 = 98.67°
theta2 = 98.67°
theta3 = 41.85°
theta1 = 98.67°
theta2 = 98.67°
theta3 = 41.85°
theta1 = 98.56°
theta2 = 98.56°
theta3 = 41.84°
theta1 = 86.99°
theta2 = 86.99°
theta3 = 41.61°
theta1 = 2.79°
theta2 = 2.79°
theta3 = 16.83°
theta1 = 5.95°
theta2 = 5.95°
theta3 = 33.09